#Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path=r'C:\Users\niels\Documents\Master Folder - Instacart Basket Analysis 09.10

### 3.Import Data 

In [3]:
#import the merged order and production data
df_customer = pd.read_csv(os.path.join(path, 'customers.csv'))

In [4]:
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### 4. Wrangle data- rename columns with illogical names and drop columns

In [5]:
df_customer.rename(columns = {'First Name': 'first_name'}, inplace=True)

In [6]:
df_customer.rename(columns = {'Surnam': 'last_name'}, inplace=True)

In [7]:
df_customer.rename(columns = {'Gender': 'gender'}, inplace=True)

In [8]:
df_customer.rename(columns = {'STATE': 'state'}, inplace=True)

In [9]:
df_customer.rename(columns = {'Age': 'age'}, inplace=True)

In [10]:
df_customer.rename(columns = {'n_dependants': 'no_of_dependents'}, inplace = True)

In [11]:
df_customer.rename(columns = {'fam_status': 'marital_status'}, inplace=True)

In [12]:
df_customer.shape

(206209, 10)

In [13]:
df_customer.head()

,user_id,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### 5. Consistency check/data quality issues (check missing / duplicate and mixed type data)

In [14]:
#check for duplicates
df_customer[df_customer.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income


In [15]:
# check for missing data
df_customer.isnull().sum()

user_id                 0
first_name          11259
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
no_of_dependents        0
marital_status          0
income                  0
dtype: int64

In [16]:
# viewing missing data
df_nan = df_customer[df_customer['first_name'].isnull() == True]

In [17]:
df_nan

,user_id,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


In [18]:
#check for missing values
df_customer.isnull().sum()

user_id                 0
first_name          11259
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
no_of_dependents        0
marital_status          0
income                  0
dtype: int64

In [19]:
#check for data types of the columns
df_customer.dtypes

user_id              int64
first_name          object
last_name           object
gender              object
state               object
age                  int64
date_joined         object
no_of_dependents     int64
marital_status      object
income               int64
dtype: object

In [20]:
#check mixed data types
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

first_name


#first_name has a mixed data type 

In [21]:
# check  data type of 'first_name'
df_customer['first_name'].dtype

dtype('O')

In [22]:
#reassign a new data type to column
df_customer['first_name'] = df_customer['first_name'].astype('str')

In [23]:
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

## 6. Combine the customer data with the rest of data

In [24]:
# Import the prepared data
df_ords_prods_merged_cus = pd.read_pickle(os.path.join(path,'ords_prods_merge_v3.pkl'))

In [27]:
#check the first few rows
df_ords_prods_merged_cus.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spending,spending_flag,purchase_frequency,frequency_flag
0,2539329,1,True,1,2,8,NaN,True,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
1,2398795,1,True,2,3,7,15.0,False,196,1,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
2,473747,1,True,3,3,12,21.0,False,196,1,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
3,2254736,1,True,4,4,7,29.0,False,196,1,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer
4,431534,1,True,5,4,15,28.0,False,196,1,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low Spender,20.5,Non-frequent customer


In [28]:
#check the shape
df_ords_prods_merged_cus.shape

(32404859, 26)

#### The common column is user_id. 

In [31]:
df_ords_prods_merged_cus['user_id'].dtype

dtype('int64')

In [33]:
# merge the two df
df_merged = df_ords_prods_merged_cus.merge(df_customer, on = 'user_id')

In [34]:
#check the new df
df_merged.shape

(32404859, 35)

In [35]:
df_merged.head()

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,...,frequency_flag,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income
0,2539329,1,True,1,2,8,NaN,True,196,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,True,2,3,7,15.0,False,196,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,True,3,3,12,21.0,False,196,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,True,4,4,7,29.0,False,196,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,True,5,4,15,28.0,False,196,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


### 7. Export as pickle file

In [36]:
df_merged.to_pickle(os.path.join(path, 'df_ords_prods_cus_merged.pkl'))